In [90]:
import pandas as pd
import numpy as np

In [39]:
import requests
import json

In [87]:
def callApi(method, stock,isEarnings = False ):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last = "/4"
    period = "period=annual"
    if isEarnings:
        url = host + ticker + api_method + last + query + period + add + key 
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [88]:
callApi("earnings", "LUV", True)

{'symbol': 'LUV',
 'earnings': [{'actualEPS': 4.27,
   'consensusEPS': 4.38,
   'announceTime': 'BTO',
   'numberOfEstimates': 20,
   'EPSSurpriseDollar': -0.11,
   'EPSReportDate': '2020-01-23',
   'fiscalPeriod': 'FY ending in 2019',
   'fiscalEndDate': '2019-12-31',
   'yearAgo': 4.24,
   'yearAgoChangePercent': 0.0071,
   'currency': None},
  {'actualEPS': 4.24,
   'announceTime': 'BTO',
   'EPSReportDate': '2019-01-24',
   'fiscalPeriod': 'FY ending in 2018',
   'fiscalEndDate': '2018-12-31',
   'yearAgo': 3.5,
   'yearAgoChangePercent': 0.2114},
  {'actualEPS': 3.5,
   'announceTime': 'BTO',
   'EPSReportDate': '2018-01-25',
   'fiscalPeriod': 'FY ending in 2017',
   'fiscalEndDate': '2017-12-31',
   'yearAgo': 3.75,
   'yearAgoChangePercent': -0.0667},
  {'actualEPS': 3.75,
   'announceTime': 'BTO',
   'EPSReportDate': '2017-01-26',
   'fiscalPeriod': 'FY ending in 2016',
   'fiscalEndDate': '2016-12-31'}]}

In [57]:
def peRatio(quote):
    if not("peRatio" in quote.keys()):
        return 0
    return quote["peRatio"]

In [66]:
def priceToBook(stats):
    if not("priceToBook" in stats.keys()):
        return 0
    return stats["priceToBook"]

In [72]:
def pegRatio(stats):
    if not("debtToEquity" in stats.keys()):
        return 0
    return stats["debtToEquity"]

In [78]:
def debtToEquity(stats):
    
    if not("pegRatio" in stats.keys()):
                return 0
    return stats["pegRatio"]

In [84]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if not("balancesheet" in balance_sheet.keys()) or len(balance_sheet["balancesheet"]) == 0:
        return 0
    latest_balance_sheet = balance_sheet["balancesheet"][0]
    if not("totalAssets" in latest_balance_sheet.keys()) and not("totalLiabilities" in latest_balance_sheet.keys()):
        return 0
    return latest_balance_sheet["totalAssets"]/ latest_balance_sheet["totalLiabilities"]
    

In [120]:
def checkErningProgression(earnings):
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return False
    earningsSet = earnings["earnings"][:-1]
    yearAgoChangePercent = np.array([])
    for earning in earningsSet:
        if not("yearAgoChangePercent" in earning.keys()):
            return False
        yearAgoChangePercent = np.append(yearAgoChangePercent, earning["yearAgoChangePercent"])
    isDecending = (yearAgoChangePercent == np.sort(yearAgoChangePercent)[::-1]).all()
    return isDecending

In [121]:
tickers = ['LUV','TELA', 'FB', 'MAR', 'JPM','HTZ', 'DAL', 'COUP']
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    data["Ticker"] = ticker
    
    # Major data set 
    quote = callApi("quote", ticker)
    stats = callApi("advanced-stats", ticker)
    balance_sheet = callApi("balance-sheet", ticker)
    earnings = callApi("earnings", ticker, True)
    
    # PE Ratio < 15
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 15"] = pe_ratio
    else: 
        data["PE Ratio < 15"] = "NA"

    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1.5"] = price_to_book
    else: 
        data["Price To Book < 1.5"] = "NA"
    
    if pe_ratio * price_to_book < 22.5 and pe_ratio * price_to_book > 0: 
        points += 1
    
    # PEG Ratio 
    peg_ratio = pegRatio(stats)
    if peg_ratio != 0: 
        data["PEG Ratio(1 and 0)"] = peg_ratio
    else: 
        data["PEG Ratio(1 and 0)"] = "NA"
    if peg_ratio < 1.1 and peg_ratio > 0:
        points += 1
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["Debt to Equity (0 and 1.5)"] = debt_Equity
    else: 
        data["Debt to Equity (0 and 1.5)"] = "NA"
    if debt_Equity < 1.6 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["Current Ratio > 1.5"] = current_ratio
    else: 
        data["Current Ratio > 1.5"] = "NA"
        
    if current_ratio > 1.4:
        points += 1
    
    # Earning Progression Last 4 year 
    isProgressing = checkErningProgression(earnings)
    data["Earning Progression"] =  isProgressing
    if isProgressing:
        points += 1
    
    data["Points"] = points
    dataSet.append(data.copy())
pd.DataFrame(dataSet)    

,Ticker,PE Ratio < 15,Price To Book < 1.5,PEG Ratio(1 and 0),Debt to Equity (0 and 1.5),Current Ratio > 1.5,Earning Progression,Points
0,LUV,96.62,2.363208,-0.33,-0.33,1.440084,False,1
1,TELA,-5.48,6.140537,0.43,0.43,2.856766,True,4
2,FB,34.29,6.726721,0.35,0.35,4.776741,False,3
3,MAR,74.81,48.356612,-0.53,-0.53,0.996933,False,0
4,JPM,13.92,1.207238,-0.27,-0.27,1.089691,False,1
5,HTZ,-0.18,0.127155,NA,NA,1.029987,True,1
6,DAL,-5.46,1.319489,0.02,0.02,1.136698,True,3
7,COUP,-213.00,43.016022,-0.38,-0.38,1.453318,True,2
